In [ ]:
import sys
print("\n".join(sys.path))

In [ ]:

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

print("\n".join(sys.path))


In [ ]:
from modeling_dandelin import VQADandelin

vqa_model = VQADandelin()

vqa_label, vqa_prob = vqa_model.detect_phone("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged/0_1.jpg", query="Is the person using a cell phone?")

In [ ]:
import torch
from transformers import ViltProcessor, ViltForQuestionAnswering
from PIL import Image

device = "cpu"
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to("cpu")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [66]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image


processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [110]:
import torch
# image = Image.open("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged/0_19.jpg")
# text = "How many cell phone are there?"
# text = "is the person using a cell phone?"
# text = "Does the person have a cell phone?"
# text = "Is there a cell phone in this image?"
# text = "Is a mobile phone visible there?"
# text = "Is a mobile phone visible in the image?"
# text = "Is a mobile phone visible in the picture?"
# text = "Is a mobile phone present in the image?"
text = "Is a cell phone present in the image?"

# images = [os.path.join("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged") ]

# images = [os.path.join("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged") ]

# images = [os.path.join("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged") if filename.startswith("0_")]


# images = [os.path.join("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332") if filename.startswith("0_")]


images = [os.path.join("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged" , filename) 
for filename in os.listdir("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged") if filename.startswith("0_")]


vqa_results = []
for image in images:
    fram_path = image
    image = Image.open(image)
    encoding = processor(image, text, return_tensors="pt")

    with torch.inference_mode():
        outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    # print("Predicted answer:", model.config.id2label[idx])
    probs = torch.softmax(logits, dim=-1)
    max_prob, _ = torch.max(probs, dim=-1)
    # print(max_prob.item())

    vqa_results.append((fram_path, model.config.id2label[idx], max_prob.item()))

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [76]:
vqa_results = sorted(vqa_results, key=lambda x: int(os.path.splitext(os.path.basename(x[0]))[0].split('_')[1]))
for result in vqa_results:
    print(result)

('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_1.jpg', 'no', 0.9999908208847046)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_2.jpg', 'no', 0.9999892711639404)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_3.jpg', 'no', 0.9999884366989136)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_4.jpg', 'no', 0.99998939037323)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_5.jpg', 'no', 0.9999922513961792)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_6.jpg', 'no', 0.9999872446060181)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_7.jpg', 'no', 0.9999880790710449)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_8.jpg', 'no', 0.9999884366989136)
('/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged/0_9.jpg', 'no', 0.999985933303833)
('/tmp/video_incidents

In [ ]:
dtype = next(model.parameters()).dtype
print(f"Current dtype of the model: {dtype}")

total_params = ( sum(p.numel() for p in model.parameters()) )
print(f"Million Parametrs: {total_params / 1000000:,}")

param_size = total_params * torch.finfo(dtype).bits / 8 
model_size_mb = param_size / (1024 ** 2)
print(f"Estimated model size: {model_size_mb:.2f} MB")

In [106]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor_blip = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model_blip = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

In [148]:
# images = [os.path.join("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged") ]

# images = [os.path.join("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged") ]

# images = [os.path.join("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged") if filename.startswith("0_")]


images = [os.path.join("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332" , filename) 
for filename in os.listdir("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332") if filename.startswith("0_")]


# images = [os.path.join("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged") if filename.startswith("0_")]


# images = images[:5]
blip_results = []
for image in images:
    fram_path = image
    raw_image = Image.open(image).convert('RGB')

    # question = "Is a cell phone clearly visible in the picture?"
    # question = "Is the person in the picture using a cell phone?"
    # question = "Is there a visible cell phone being used by the person in the image?"
    # question = "Is there a cell phone clearly visible and being used by the person in the image?"
    # question = "Is there a cell phone clearly present and being used by the person in the image?"
    # question = "Is there a cell phone clearly visible in the image where a person is present?"
    # question = "Is there a cell phone clearly visible in the image and a person is present?"
    # question = "Is there a cell phone visible in the image, with a person present?"
    # question = "Is there a cell phone visible in the image, with a person present?"
    # question = "Is there a cell phone clearly present in the picture?"
    # question = "Is there a cell phone present in the picture, and is it being held by a person?"
    # question = "Is there a cell phone held by a person in the picture?"
    # question = "Is the person in the picture holding a phone in their hand?"
    # question = "Is the person in the picture holding a cell phone in their hand?"
    # question = "Is a cell phone being held in hand by the person in the picture?"
    # question = "Is a cell phone in the person's hand?"
    question = "Is a cell phone clearly visible in the person's hand?"

    inputs = processor_blip(raw_image, question, return_tensors="pt")

    with torch.inference_mode():
        out = model_blip.generate(**inputs)
    real_output = processor_blip.decode(out[0], skip_special_tokens=True)
    # print(real_output)

    blip_results.append((fram_path, real_output))

In [149]:
blip_results = sorted(blip_results, key=lambda x: int(os.path.splitext(os.path.basename(x[0]))[0].split('_')[1]))

for result in blip_results:
    print(result)

('/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332/0_1.jpg', 'no')
('/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332/0_2.jpg', 'no')
('/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332/0_3.jpg', 'no')
('/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332/0_4.jpg', 'no')
('/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332/0_5.jpg', 'no')
('/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332/0_6.jpg', 'no')
('/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332/0_7.jpg', 'no')
('/tmp/video_inciden

In [ ]:
dtype = next(model_blip.parameters()).dtype
print(f"Current dtype of the model: {dtype}")

total_params = ( sum(p.numel() for p in model_blip.parameters()) )
print(f"Million Parametrs: {total_params / 1000000:,}")

param_size = total_params * torch.finfo(dtype).bits / 8 
model_size_mb = param_size / (1024 ** 2)
print(f"Estimated model size: {model_size_mb:.2f} MB")

In [ ]:
from modeling_yolov8 import YOLOv8PersonDetector
import os
import cv2
from ultralytics import YOLO
import torch
import time

yolov8_persondetector = YOLOv8PersonDetector()

model_yolo = YOLO("yolov8n.pt").to("cpu")
classes_to_detect = [67]  # Class 0 for "person" in COCO dataset

In [ ]:
results = model_yolo.predict(["/home/ajeet/Downloads/cropped_phone.jpg"], conf=0.0, 
                             device="cpu", imgsz=320, classes=classes_to_detect, 
                        save=True, save_txt=True, show_labels=True, show_conf=True, show_boxes=True,iou=0.40)

In [ ]:
print(next(model_yolo.parameters()).dtype)

total_params = (sum(p.numel() for p in model_yolo.parameters())) 
print(f"Million Parametrs: {total_params / 1000000:,}")

param_size = total_params * 32 / 8 
model_size_mb = param_size / (1024 ** 2)
print(f"Estimated model size: {model_size_mb :.2f} MB")

In [ ]:
dtype = next(model_blip.parameters()).dtype
print(f"Current dtype of the model: {dtype}")

total_params = ( sum(p.numel() for p in model_blip.parameters()) )
print(f"Million Parametrs: {total_params / 1000000:,}")

param_size = total_params * torch.finfo(dtype).bits / 8 
model_size_mb = param_size / (1024 ** 2)
print(f"Estimated model size: {model_size_mb:.2f} MB")

In [ ]:
from transformers import BlipForQuestionAnswering, BlipProcessor, AutoModelForQuestionAnswering
from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # or use load_in_8bit for 8-bit quantization
    # bnb_4bit_compute_dtype=torch.float16,  # Compute in FP164
    # bnb_4bit_use_double_quant=True,  # Double quantization for improved accuracy
    # bnb_4bit_quant_type="nf4"  # Use NormalFloat4 quantization
)

# Load the model with the quantization configuration
model_name = "Salesforce/blip-vqa-base"
model_quantization = BlipForQuestionAnswering.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="cpu"
)

processor_quantization = BlipProcessor.from_pretrained(model_name)

In [ ]:
dtype = next(model_blip.parameters()).dtype
print(f"Current dtype of the model: {dtype}")

total_params = ( sum(p.numel() for p in model_blip.parameters()) )
print(f"Million Parametrs: {total_params / 1000000:,}")

param_size = total_params * torch.finfo(dtype).bits / 8 
model_size_mb = param_size / (1024 ** 2)
print(f"Estimated model size: {model_size_mb:.2f} MB")

In [22]:
# Ultralytics YOLO 🚀, AGPL-3.0 license

import argparse

import cv2
import numpy as np
import onnxruntime as ort
import torch
import os

from ultralytics.utils import ASSETS, yaml_load
from ultralytics.utils.checks import check_requirements, check_yaml


class YOLOv8:
    """YOLOv8 object detection model class for handling inference and visualization."""

    def __init__(self, onnx_model, input_image, confidence_thres, iou_thres):
        """
        Initializes an instance of the YOLOv8 class.

        Args:
            onnx_model: Path to the ONNX model.
            input_image: Path to the input image.
            confidence_thres: Confidence threshold for filtering detections.
            iou_thres: IoU (Intersection over Union) threshold for non-maximum suppression.
        """
        self.onnx_model = onnx_model
        self.input_image = input_image
        self.confidence_thres = confidence_thres
        self.iou_thres = iou_thres

        # Load the class names from the COCO dataset
        self.classes = yaml_load(check_yaml("coco8.yaml"))["names"]

        # Generate a color palette for the classes
        self.color_palette = np.random.uniform(0, 255, size=(len(self.classes), 3))

    def draw_detections(self, img, box, score, class_id):
        """
        Draws bounding boxes and labels on the input image based on the detected objects.

        Args:
            img: The input image to draw detections on.
            box: Detected bounding box.
            score: Corresponding detection score.
            class_id: Class ID for the detected object.

        Returns:
            None
        """
        # Extract the coordinates of the bounding box
        x1, y1, w, h = box

        # Retrieve the color for the class ID
        color = self.color_palette[class_id]

        # Draw the bounding box on the image
        cv2.rectangle(img, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

        # Create the label text with class name and score
        label = f"{self.classes[class_id]}: {score:.2f}"

        # Calculate the dimensions of the label text
        (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

        # Calculate the position of the label text
        label_x = x1
        label_y = y1 - 10 if y1 - 10 > label_height else y1 + 10

        # Draw a filled rectangle as the background for the label text
        cv2.rectangle(
            img, (label_x, label_y - label_height), (label_x + label_width, label_y + label_height), color, cv2.FILLED
        )

        # Draw the label text on the image
        cv2.putText(img, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

    def preprocess(self):
        """
        Preprocesses the input image before performing inference.

        Returns:
            image_data: Preprocessed image data ready for inference.
        """
        # Read the input image using OpenCV
        self.img = cv2.imread(self.input_image)

        # Get the height and width of the input image
        self.img_height, self.img_width = self.img.shape[:2]

        # Convert the image color space from BGR to RGB
        img = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)

        # Resize the image to match the input shape
        img = cv2.resize(img, (self.input_width, self.input_height))

        # Normalize the image data by dividing it by 255.0
        image_data = np.array(img) / 255.0

        # Transpose the image to have the channel dimension as the first dimension
        image_data = np.transpose(image_data, (2, 0, 1))  # Channel first

        # Expand the dimensions of the image data to match the expected input shape
        image_data = np.expand_dims(image_data, axis=0).astype(np.float32)

        # Return the preprocessed image data
        return image_data

    def postprocess(self, input_image, output):
        """
        Performs post-processing on the model's output to extract bounding boxes, scores, and class IDs.

        Args:
            input_image (numpy.ndarray): The input image.
            output (numpy.ndarray): The output of the model.

        Returns:
            numpy.ndarray: The input image with detections drawn on it.
        """
        # Transpose and squeeze the output to match the expected shape
        outputs = np.transpose(np.squeeze(output[0]))

        # Get the number of rows in the outputs array
        rows = outputs.shape[0]

        # Lists to store the bounding boxes, scores, and class IDs of the detections
        boxes = []
        scores = []
        class_ids = []
        phone_detected = False

        # Calculate the scaling factors for the bounding box coordinates
        x_factor = self.img_width / self.input_width
        y_factor = self.img_height / self.input_height

        # Iterate over each row in the outputs array
        for i in range(rows):
            # Extract the class scores from the current row
            classes_scores = outputs[i][4:]
            # print(classes_scores)

            # num_classes = len(classes_scores)
            # print(num_classes)

        # # Check if the phone class (assumed ID 67) is within the range of class IDs
        #     if 67 < num_classes:
        #         phone_score = classes_scores[67]
        #         if phone_score >= self.confidence_thres:
        #             phone_detected = True

        #             # Extract the bounding box coordinates from the current row
        #             x, y, w, h = outputs[i][0], outputs[i][1], outputs[i][2], outputs[i][3]

        #             # Calculate the scaled coordinates of the bounding box
        #             left = int((x - w / 2) * x_factor)
        #             top = int((y - h / 2) * y_factor)
        #             width = int(w * x_factor)
        #             height = int(h * y_factor)

        #             # Add the class ID (phone), score, and box coordinates to the respective lists
        #             class_ids.append(67)  # Phone class ID
        #             scores.append(phone_score)
        #         boxes.append([left, top, width, height])

            # Find the maximum score among the class scores
            max_score = np.amax(classes_scores)
            # print(max_score)

            # If the maximum score is above the confidence threshold
            if max_score >= self.confidence_thres:
                # Get the class ID with the highest score
                class_id = np.argmax(classes_scores)
                # print(f"class_id: {class_id}")

                if class_id == 2:
                    phone_detected = True
                    # Extract the bounding box coordinates from the current row
                    x, y, w, h = outputs[i][0], outputs[i][1], outputs[i][2], outputs[i][3]

                    # Calculate the scaled coordinates of the bounding box
                    left = int((x - w / 2) * x_factor)
                    top = int((y - h / 2) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    # Add the class ID, score, and box coordinates to the respective lists
                    class_ids.append(class_id)
                    scores.append(max_score)
                    boxes.append([left, top, width, height])

        # Apply non-maximum suppression to filter out overlapping bounding boxes
        indices = cv2.dnn.NMSBoxes(boxes, scores, self.confidence_thres, self.iou_thres)

        # Iterate over the selected indices after non-maximum suppression
        for i in indices:
            # Get the box, score, and class ID corresponding to the index
            box = boxes[i]
            score = scores[i]
            class_id = class_ids[i]

            # Draw the detection on the input image
            self.draw_detections(input_image, box, score, class_id)

        # Return the modified input image
        return input_image , phone_detected

    def main(self):
        """
        Performs inference using an ONNX model and returns the output image with drawn detections.

        Returns:
            output_img: The output image with drawn detections.
        """
        # Create an inference session using the ONNX model and specify execution providers
        session = ort.InferenceSession(self.onnx_model, providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

        # Get the model inputs
        model_inputs = session.get_inputs()

        # Store the shape of the input for later use
        input_shape = model_inputs[0].shape
        self.input_width = input_shape[2]
        self.input_height = input_shape[3]

        # Preprocess the image data
        img_data = self.preprocess()

        # Run inference using the preprocessed image data
        outputs = session.run(None, {model_inputs[0].name: img_data})

        # Perform post-processing on the outputs to obtain output image.
        return self.postprocess(self.img, outputs) # output image

def save_image(image, path, image_name):
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(os.path.join(path, image_name), image)

if __name__ == "__main__":
    # Create an argument parser to handle command-line arguments
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--model", type=str, default="yolov8n.onnx", help="Input your ONNX model.")
    # parser.add_argument("--img", type=str, default=str(ASSETS / "bus.jpg"), help="Path to input image.")
    # parser.add_argument("--conf-thres", type=float, default=0.5, help="Confidence threshold")
    # parser.add_argument("--iou-thres", type=float, default=0.5, help="NMS IoU threshold")
    # args = parser.parse_args()

    # Check the requirements and select the appropriate backend (CPU or GPU)
    # check_requirements("onnxruntime-gpu" if torch.cuda.is_available() else "onnxruntime")

    # Create an instance of the YOLOv8 class with the specified arguments
    # detection = YOLOv8(args.model, args.img, args.conf_thres, args.iou_thres)
    # images = [os.path.join("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged" , filename) 
    # for filename in os.listdir("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged") ]


    # images = [os.path.join("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged" , filename) 
    # for filename in os.listdir("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged") ]

    # images = [os.path.join("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged" , filename) 
    # for filename in os.listdir("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged") ]

    # images = [os.path.join("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged" , filename) 
    # for filename in os.listdir("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged") if filename.startswith("0_")]


    # images = [os.path.join("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332" , filename) 
    # for filename in os.listdir("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332") if filename.startswith("0_")]


    # images = [os.path.join("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged" , filename) 
    # for filename in os.listdir("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged") if filename.startswith("0_")]


    # images = images[:1]

    # images = sorted(images, key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[1]))

    images = [os.path.join("/home/ajeet/codework/Cellphone_train/train" , filename) 
    for filename in os.listdir("/home/ajeet/codework/Cellphone_train/train")]

    images = images[:1000]

    for image in images:
        detection = YOLOv8("/home/ajeet/Downloads/od_v8_nano_feb24_2.onnx",image, 0.001, 0.40)

        # Perform object detection and obtain the output image
        output_image, phone_detected = detection.main()
        
        if phone_detected:
            print(f"{image}: phone_detected ")
        else:
            print(f"{image}: phone_not_detected ")



        save_path_with_phone = "/home/ajeet/codework/finetuned_yolo"
        image_name = os.path.basename(detection.input_image)
        save_image(output_image, save_path_with_phone, image_name)

        # Display the output image in a window
        # cv2.namedWindow("Output", cv2.WINDOW_NORMAL)
        # cv2.imshow("Output", output_image)

        # # Wait for a key press to exit
        # cv2.waitKey(10000)
        # cv2.destroyAllWindows()

/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_1.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_2.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_3.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_4.jpg: phone_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_5.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_6.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_7.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_8.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_9.jpg: phone_not_detected 
/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_10.jpg: phone_not_detected 
/tm

In [4]:
import onnxruntime as ort

# Load the ONNX model
onnx_model_path = "/home/ajeet/Downloads/od_v8_nano_feb24_2.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

# Get the model inputs
model_inputs = session.get_inputs()
for i, input in enumerate(model_inputs):
    print(f"Input {i}:")
    print(f"  Name: {input.name}")
    print(f"  Shape: {input.shape}")
    # print(f"  Type: {input.dtype}")

# Get the model outputs
model_outputs = session.get_outputs()
for i, output in enumerate(model_outputs):
    print(f"Output {i}:")
    print(f"  Name: {output.name}")
    print(f"  Shape: {output.shape}")
    # print(f"  Type: {output.dtype}")

Input 0:
  Name: images
  Shape: [1, 3, 640, 640]
Output 0:
  Name: output0
  Shape: [1, 8, 8400]


/home/ajeet/codework/venv_visionwork/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [ ]:
from modeling_clip import CLIP
import time

clip_model = CLIP(device="cpu")

text_prompts = {
"phone": [
    "a photo of a person using a cell phone."

],
"no_phone": [
    "a photo of a person not using a cell phone."
]
}

images = [os.path.join("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged" , filename) 
for filename in os.listdir("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged") ]

# images = images[:10]

images = sorted(images, key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[1]))

start_time = time.time()
_, image_embeddings = clip_model.get_image_embeddings(images)
time_taken_by_embeddings = time.time() - start_time
print(f"Time taken to extract embeddings: {time_taken_by_embeddings:.2f} seconds")

precomputed_embeddings = clip_model.precompute_prompt_embeddings(text_prompts)

In [ ]:
clip_results = []
for frame_path, image_embedding in zip(images, image_embeddings):
    clip_label, clip_prob = clip_model.classify_image(image_embedding, precomputed_embeddings)
    clip_prob = 1
    clip_results.append((frame_path, clip_label, clip_prob))  # Store frame path, label, and probability

for result in clip_results:
    print(result)

In [ ]:
import numpy as np

def cosine_similarity_to_probability(cosine_similarity, k=5, b=0):
    # Apply the sigmoid function
    probability = 1 / (1 + np.exp(-k * (cosine_similarity - b)))
    return probability

# Example usage
cosine_similarity = 0.25  # Example cosine similarity value
probability = cosine_similarity_to_probability(cosine_similarity)
print(f"Probability: {probability}")


In [1]:
from PIL import Image
import requests
import torch
from transformers import CLIPProcessor, CLIPModel
import os

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/home/ajeet/codework/venv_visionwork/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-16 18:42:48.064939: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-16 18:42:48.100295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 18:42:48.779930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
frame_path = "/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged/0_142.jpg"
image = Image.open(frame_path)
list_of_classes = ["a person", "a cell phone"]
inputs = processor(text=list_of_classes, images=image, return_tensors="pt", padding=True)

with torch.inference_mode():
    outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

# # print(probs)
# max_prob_index = int(torch.argmax(probs))
# # print(list_of_classes[max_prob_index])


logits_per_image = outputs.logits_per_image # image-text similarity score
probs = logits_per_image.softmax(dim=1) # label probabilities
max_prob_index = int(torch.argmax(probs))

print((frame_path, list_of_classes[max_prob_index], probs))

('/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged/0_142.jpg', 'a cell phone', tensor([[0.4953, 0.5047]]))


In [103]:
# images = [os.path.join("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/9d47d182-296b-47d3-a64f-d6a2fd06f9f3_merged") ]

# images = [os.path.join("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/f044cf1d-8a5b-4703-a05c-3b57c5c14989_merged") ]

images = [os.path.join("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged" , filename) 
for filename in os.listdir("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged") if filename.startswith("0_")]


# images = [os.path.join("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/ed2e9e4c-ec0e-437d-a13d-5a1b87340c44_20240911181755971334_20240911181756999046_0_merged_140332") if filename.startswith("0_")]


# images = [os.path.join("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged" , filename) 
# for filename in os.listdir("/tmp/video_incidents_ajeet/9aa3bbb8-9cc2-48a9-a2a6-aa9c86acf71b_20240911182617744144_20240911182619026111_0_merged") if filename.startswith("0_")]


# images = images[:10]

images = sorted(images, key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[1]))

# Define the list of classes 
# list_of_classes = ['human', "phone"]
list_of_classes = ["person", "cell phone"]
clip_prob_list = []
for image in images:
    frame_path = image
    image = Image.open(image)

    inputs = processor(text=list_of_classes, images=image, return_tensors="pt", padding=True)

    with torch.inference_mode():
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

    # print(probs)
    max_prob_index = int(torch.argmax(probs))
    # print(list_of_classes[max_prob_index])

    clip_prob_list.append((frame_path, list_of_classes[max_prob_index], probs))

In [104]:
for result in clip_prob_list:
    print(result)
    # my_tensor = result[2]
    # if my_tensor[0][1] > 0.90:
    #     print(result)

('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_1.jpg', 'person', tensor([[0.8162, 0.1838]]))
('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_2.jpg', 'person', tensor([[0.7666, 0.2334]]))
('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_3.jpg', 'person', tensor([[0.7783, 0.2217]]))
('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_4.jpg', 'person', tensor([[0.7406, 0.2594]]))
('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_5.jpg', 'person', tensor([[0.8481, 0.1519]]))
('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_6.jpg', 'person', tensor([[0.8833, 0.1167]]))
('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_7.jpg', 'person', tensor([[0.8572, 0.1428]]))
('/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_8.jpg', 'person', tensor([[0.8464, 0.1536]]))
('/tmp/video_incidents_ajeet/b20

In [8]:
images = [os.path.join("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged" , filename) 
for filename in os.listdir("/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged") if filename.startswith("0_")]

# images = images[:10]

images = sorted(images, key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[1]))

In [9]:
images

['/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_1.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_2.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_3.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_4.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_5.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_6.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_7.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_8.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_9.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_10.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_11.jpg',
 '/tmp/video_incidents_ajeet/b2045fbc-5af3-4b40-a166-072b90f803e5_merged/0_12.jpg',
 